# Serverless

In [6]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.applications.xception import preprocess_input

In [2]:
!wget https://github.com/alexeygrigorev/mlbookcamp-code/releases/download/chapter7-model/xception_v4_large_08_0.894.h5 -O clothing_model.h5 --quiet

In [4]:
model = keras.models.load_model('clothing_model.h5')

In [5]:
!wget http://bit.ly/mlbookcamp-pants -O pants.jpg --quiet

In [7]:
img = load_img('pants.jpg', target_size=(299, 299))

In [8]:
x = np.array(img)
X = np.array([x])
X = preprocess_input(X)

In [9]:
X.shape

(1, 299, 299, 3)